**With VS Code Jupyter you don't need to add the kernel to Jupyter Notebook like before to have it as an environment**

In [1]:
import polars as pl
import json
from tqdm.auto import tqdm

/users/labnet5/gr5/abahari/Documents/Thesis/src/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./params.json", mode = "r", encoding = "utf-8") as f:
    data = json.load(f)
    model_path = data["model_path"]
    num_single_sample_timesteps = data["num_single_sample_timesteps"]
    window_stride = data["window_stride"]
    input_window_length = data["input_window_length"]
    label_window_length = data["label_window_length"]
    input_features = data["input_features"]
    label_features = data["label_features"]
    positional_encoding_max_len = data["positional_encoding_max_len"]
    embedding_dim = data["embedding_dim"]
    num_attention_head = data["num_attention_head"]
    num_encoder_layers = data["num_encoder_layers"]
    num_decoder_layers = data["num_decoder_layers"]
    position_wise_nn_dim = data["position_wise_nn_dim"]
    dropout = data["dropout"]
    batch_size = data["batch_size"]
    epochs = data["epochs"]
    learning_rate = data["learning_rate"]

In [25]:
df = pl.read_csv("./reversalData_minor.csv")
df

id,eps,n_0_squared,psi_e,b_e,psi_plus,b_plus,u_list,r_list,k_e_psi_e_list,k_e_b_e_list,k_e_psi_plus_list,k_e_b_plus_list,heat_flux_psi_e_b_e_list,heat_flux_psi_e_b_plus_list,b_e_psi_plus_list,b_e_b_plus_list,psi_plus_b_plus_list,eta_list
i64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1500,0.123943,318.864022,"""[-0.011744609035812982, -0.011…","""[-5.517162102724528, -3.192070…","""[-0.0016475367466768115, -0.00…","""[10.73076649947905, 10.7758112…","""[0.4615413218450946, 0.4686702…","""[0.6551578220909112, 0.8945764…","""[0.00013793584140409996, 0.000…","""[30.43907766773974, 10.1893158…","""[2.7143773316504123e-06, 5.549…","""[115.14934966634185, 116.11810…","""[0.06479691188370344, 0.035800…","""[-0.1260286571909809, -0.12085…","""[0.009089727301611367, 0.00751…","""[-59.203378264111755, -34.3971…","""[-0.01767933212790023, -0.0253…","""[[1.0369206541904592], [-1.681…"
1501,0.123943,318.864022,"""[-0.008147157474100065, -0.008…","""[-0.832904729882089, 0.1407146…","""[0.0006807256129005129, 0.0007…","""[-0.7190970906032216, -0.60951…","""[0.23703131704960073, 0.234794…","""[-0.18778048603775238, -0.2053…","""[6.637617490778455e-05, 6.8758…","""[0.6937302890599556, 0.0198006…","""[4.6338736005877887e-07, 5.349…","""[0.5171006257140178, 0.3715125…","""[0.006785805995272157, -0.0011…","""[0.005858597236311648, 0.00505…","""[-0.0005669795827367211, 0.000…","""[0.5989393680078723, -0.085768…","""[-0.0004895078077358536, -0.00…","""[[1.418213787060706], [0.88278…"
1502,0.123943,318.864022,"""[-0.003781434990007615, -0.001…","""[7.429632504232679, 7.61617440…","""[0.0017201097552821953, 0.0016…","""[0.26192796713213556, -0.01619…","""[0.3398928494975913, 0.3384786…","""[-0.22023434840890527, -0.0752…","""[1.4299250583653892e-05, 1.712…","""[55.199439147950756, 58.006112…","""[2.958777570216974e-06, 2.8808…","""[0.06860625996597308, 0.000262…","""[-0.028094672314403355, -0.009…","""[-0.000990463579775022, 2.1190…","""[0.012779783348692317, 0.01292…","""[1.9460285383725033, -0.123315…","""[0.00045054485144522063, -2.74…","""[[-0.6679438799496836], [-0.80…"
1503,0.123943,318.864022,"""[-0.008102021680746106, -0.007…","""[-13.405475487140006, -12.1429…","""[-0.007065355563569785, -0.007…","""[3.432058579711073, 2.92797364…","""[0.15021463519080008, 0.170829…","""[1.9382048681079675, 1.8501503…","""[6.564275531527996e-05, 5.7415…","""[179.70677303631157, 147.45183…","""[4.991924923966652e-05, 5.2004…","""[11.779026094568387, 8.5730296…","""[0.10861145303751879, 0.092010…","""[-0.0278066130224098, -0.02218…","""[0.09471445081536302, 0.087568…","""[-46.00837716074533, -35.55430…","""[-0.024248714180659044, -0.021…","""[[0.20668630436664126], [1.034…"
1504,0.123943,318.864022,"""[-0.0007052290130828054, -0.00…","""[7.188510348955567, 7.25531527…","""[0.01155266108186768, 0.011468…","""[-0.9665805895951766, 0.236189…","""[0.1388431307750241, 0.1327890…","""[-0.2758572865676108, -1.34969…","""[4.973479608937477e-07, 1.2082…","""[51.67468103704128, 52.6395997…","""[0.0001334639780725001, 0.0001…","""[0.9342780361821591, 0.0557856…","""[-0.0050695460589294675, -0.02…","""[0.0006816606752652025, -0.000…","""[0.08304642374498203, 0.083205…","""[-6.9482745714045, 1.713632080…","""[-0.011166577959904913, 0.0027…","""[[-0.2944195173353409], [0.846…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2995,0.123943,318.864022,"""[0.0027304602552334, -0.002678…","""[-15.816264644897496, -15.5200…","""[0.0012142394002116878, 0.0009…","""[2.764721975700982, 4.05941343…","""[0.4027182522813478, 0.4025100…","""[0.11225674279144691, -0.08758…","""[7.455413205409244e-06, 7.1762…","""[250.1542273174345, 240.873016…","""[1.4743773210264393e-06, 9.324…","""[7.643687602923941, 16.4788374…","""[-0.04318568199914582, 0.04157…","""[0.007548963471421893, -0.0108…","""[-0.01920473169600966, -0.0149…","""[-43.727574437250595, -63.0024…","""[0.0033570343535272327, 0.0039…","""[[-1.3447105080916295], [-0.78…"
2996,0.123943,318.864022,"""[0.00381980078

In [26]:
df.shape

(1500, 19)

In [27]:
df = df.drop(["id", "eps", "n_0_squared"])
df.head()

psi_e,b_e,psi_plus,b_plus,u_list,r_list,k_e_psi_e_list,k_e_b_e_list,k_e_psi_plus_list,k_e_b_plus_list,heat_flux_psi_e_b_e_list,heat_flux_psi_e_b_plus_list,b_e_psi_plus_list,b_e_b_plus_list,psi_plus_b_plus_list,eta_list
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""[-0.011744609035812982, -0.011…","""[-5.517162102724528, -3.192070…","""[-0.0016475367466768115, -0.00…","""[10.73076649947905, 10.7758112…","""[0.4615413218450946, 0.4686702…","""[0.6551578220909112, 0.8945764…","""[0.00013793584140409996, 0.000…","""[30.43907766773974, 10.1893158…","""[2.7143773316504123e-06, 5.549…","""[115.14934966634185, 116.11810…","""[0.06479691188370344, 0.035800…","""[-0.1260286571909809, -0.12085…","""[0.009089727301611367, 0.00751…","""[-59.203378264111755, -34.3971…","""[-0.01767933212790023, -0.0253…","""[[1.0369206541904592], [-1.681…"
"""[-0.008147157474100065, -0.008…","""[-0.832904729882089, 0.1407146…","""[0.0006807256129005129, 0.0007…","""[-0.7190970906032216, -0.60951…","""[0.23703131704960073, 0.234794…","""[-0.18778048603775238, -0.2053…","""[6.637617490778455e-05, 6.8758…","""[0.6937302890599556, 0.0198006…","""[4.6338736005877887e-07, 5.349…","""[0.5171006257140178, 0.3715125…","""[0.006785805995272157, -0.0011…","""[0.005858597236311648, 0.00505…","""[-0.0005669795827367211, 0.000…","""[0.5989393680078723, -0.085768…","""[-0.0004895078077358536, -0.00…","""[[1.418213787060706], [0.88278…"
"""[-0.003781434990007615, -0.001…","""[7.429632504232679, 7.61617440…","""[0.0017201097552821953, 0.0016…","""[0.26192796713213556, -0.01619…","""[0.3398928494975913, 0.3384786…","""[-0.22023434840890527, -0.0752…","""[1.4299250583653892e-05, 1.712…","""[55.199439147950756, 58.006112…","""[2.958777570216974e-06, 2.8808…","""[0.06860625996597308, 0.000262…","""[-0.028094672314403355, -0.009…","""[-0.000990463579775022, 2.1190…","""[0.012779783348692317, 0.01292…","""[1.9460285383725033, -0.123315…","""[0.00045054485144522063, -2.74…","""[[-0.6679438799496836], [-0.80…"
"""[-0.008102021680746106, -0.007…","""[-13.405475487140006, -12.1429…","""[-0.007065355563569785, -0.007…","""[3.432058579711073, 2.92797364…","""[0.15021463519080008, 0.170829…","""[1.9382048681079675, 1.8501503…","""[6.564275531527996e-05, 5.7415…","""[179.70677303631157, 147.45183…","""[4.991924923966652e-05, 5.2004…","""[11.779026094568387, 8.5730296…","""[0.10861145303751879, 0.092010…","""[-0.0278066130224098, -0.02218…","""[0.09471445081536302, 0.087568…","""[-46.00837716074533, -35.55430…","""[-0.024248714180659044, -0.021…","""[[0.20668630436664126], [1.034…"
"""[-0.0007052290130828054, -0.00…","""[7.188510348955567, 7.25531527…","""[0.01155266108186768, 0.011468…","""[-0.9665805895951766, 0.236189…","""[0.1388431307750241, 0.1327890…","""[-0.2758572865676108, -1.34969…","""[4.973479608937477e-07, 1.2082…","""[51.67468103704128, 52.6395997…","""[0.0001334639780725001, 0.0001…","""[0.9342780361821591, 0.0557856…","""[-0.0050695460589294675, -0.02…","""[0.0006816606752652025, -0.000…","""[0.08304642374498203, 0.083205…","""[-6.9482745714045, 1.713632080…","""[-0.011166577959904913, 0.0027…","""[[-0.2944195173353409], [0.846…"


In [28]:
df = df.select(
    pl.col("*").str.json_decode()
)
df

psi_e,b_e,psi_plus,b_plus,u_list,r_list,k_e_psi_e_list,k_e_b_e_list,k_e_psi_plus_list,k_e_b_plus_list,heat_flux_psi_e_b_e_list,heat_flux_psi_e_b_plus_list,b_e_psi_plus_list,b_e_b_plus_list,psi_plus_b_plus_list,eta_list
list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[list[f64]]
"[-0.011745, -0.011215, … -0.011728]","[-5.517162, -3.192071, … 6.209561]","[-0.001648, -0.002356, … 0.000093]","[10.730766, 10.775811, … 2.362047]","[0.461541, 0.46867, … -0.278246]","[0.655158, 0.894576, … -0.036835]","[0.000138, 0.000126, … 0.000138]","[30.439078, 10.189316, … 38.558648]","[0.000003, 0.000006, … 8.6049e-9]","[115.14935, 116.118109, … 5.579266]","[0.064797, 0.0358, … -0.072824]","[-0.126029, -0.120854, … -0.027701]","[0.00909, 0.00752, … 0.000576]","[-59.203378, -34.397152, … 14.667275]","[-0.017679, -0.025385, … 0.000219]","[[1.036921], [-1.681305], … [-0.782573]]"
"[-0.008147, -0.008292, … -0.004894]","[-0.832905, 0.140715, … -8.502144]","[0.000681, 0.000731, … 0.006186]","[-0.719097, -0.609518, … -5.884718]","[0.237031, 0.234794, … -0.347979]","[-0.18778, -0.20534, … -1.025089]","[0.000066, 0.000069, … 0.000024]","[0.69373, 0.019801, … 72.28645]","[4.6339e-7, 5.3490e-7, … 0.000038]","[0.517101, 0.371513, … 34.629909]","[0.006786, -0.001167, … 0.041612]","[0.005859, 0.005054, … 0.028802]","[-0.000567, 0.000103, … -0.052593]","[0.598939, -0.085768, … 50.032721]","[-0.00049, -0.000446, … -0.036402]","[[1.418214], [0.882783], … [-0.412105]]"
"[-0.003781, -0.001309, … 0.004499]","[7.429633, 7.616174, … 0.77588]","[0.00172, 0.001697, … 0.001258]","[0.261928, -0.016191, … -5.434772]","[0.339893, 0.338479, … -0.232213]","[-0.220234, -0.075214, … 0.191679]","[0.000014, 0.000002, … 0.00002]","[55.199439, 58.006113, … 0.60199]","[0.000003, 0.000003, … 0.000002]","[0.068606, 0.000262, … 29.53675]","[-0.028095, -0.009968, … 0.003491]","[-0.00099, 0.000021, … -0.024451]","[0.01278, 0.012927, … 0.000976]","[1.946029, -0.123316, … -4.216733]","[0.000451, -0.000027, … -0.006839]","[[-0.667944], [-0.804122], … [0.820629]]"
"[-0.008102, -0.007577, … 0.016667]","[-13.405475, -12.142975, … 11.825626]","[-0.007065, -0.007211, … -0.00118]","[3.432059, 2.927974, … 3.577291]","[0.150215, 0.17083, … -0.121867]","[1.938205, 1.85015, … -0.665716]","[0.000066, 0.000057, … 0.000278]","[179.706773, 147.451835, … 139.845423]","[0.00005, 0.000052, … 0.000001]","[11.779026, 8.57303, … 12.797011]","[0.108611, 0.092011, … 0.1971]","[-0.027807, -0.022186, … 0.059623]","[0.094714, 0.087568, … -0.01395]","[-46.008377, -35.55431, … 42.303705]","[-0.024249, -0.021115, … -0.00422]","[[0.206686], [1.034311], … [-2.368951]]"
"[-0.000705, -0.003476, … 0.004359]","[7.18851, 7.255315, … -1.118738]","[0.011553, 0.011468, … 0.001921]","[-0.966581, 0.23619, … 0.077109]","[0.138843, 0.132789, … -0.237516]","[-0.275857, -1.349696, … 0.283456]","[4.9735e-7, 0.000012, … 0.000019]","[51.674681, 52.6396, … 1.251575]","[0.000133, 0.000132, … 0.000004]","[0.934278, 0.055786, … 0.005946]","[-0.00507, -0.025219, … -0.004877]","[0.000682, -0.000821, … 0.000336]","[0.083046, 0.083205, … -0.002149]","[-6.948275, 1.713632, … -0.086265]","[-0.011167, 0.002709, … 0.000148]","[[-0.29442], [0.846611], … [0.437807]]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"[0.00273, -0.002679, … -0.012394]","[-15.816265, -15.520084, … -10.815311]","[0.001214, 0.000966, … 0.00499]","[2.764722, 4.059413, … -10.471988]","[0.402718, 0.40251, … -0.385353]","[0.112257, -0.087583, … -2.093855]","[0.000007, 0.000007, … 0.000154]","[250.154227, 240.873016, … 116.970942]","[0.000001, 9.3240e-7, … 0.000025]","[7.643688, 16.478837, … 109.662526]","[-0.043186, 0.041576, … 0.13404]","[0.007549, -0.010875, … 0.129785]","[-0.019205, -0.014986, … -0.053966]","[-43.727574, -63.002439, … 113.257799]","[0.003357, 0.00392, … -0.052253]","[[-1.344711], [-0.787466], … [-0.490276]]"
"[0.00382, 0.001066, … -0.010784]","[-6.111776, -6.408017, …

In [29]:
df = df.with_columns(
    eta_list = pl.col("eta_list").list.eval(pl.element().flatten(), parallel = True)
)
df

psi_e,b_e,psi_plus,b_plus,u_list,r_list,k_e_psi_e_list,k_e_b_e_list,k_e_psi_plus_list,k_e_b_plus_list,heat_flux_psi_e_b_e_list,heat_flux_psi_e_b_plus_list,b_e_psi_plus_list,b_e_b_plus_list,psi_plus_b_plus_list,eta_list
list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64]
"[-0.011745, -0.011215, … -0.011728]","[-5.517162, -3.192071, … 6.209561]","[-0.001648, -0.002356, … 0.000093]","[10.730766, 10.775811, … 2.362047]","[0.461541, 0.46867, … -0.278246]","[0.655158, 0.894576, … -0.036835]","[0.000138, 0.000126, … 0.000138]","[30.439078, 10.189316, … 38.558648]","[0.000003, 0.000006, … 8.6049e-9]","[115.14935, 116.118109, … 5.579266]","[0.064797, 0.0358, … -0.072824]","[-0.126029, -0.120854, … -0.027701]","[0.00909, 0.00752, … 0.000576]","[-59.203378, -34.397152, … 14.667275]","[-0.017679, -0.025385, … 0.000219]","[1.036921, -1.681305, … -0.782573]"
"[-0.008147, -0.008292, … -0.004894]","[-0.832905, 0.140715, … -8.502144]","[0.000681, 0.000731, … 0.006186]","[-0.719097, -0.609518, … -5.884718]","[0.237031, 0.234794, … -0.347979]","[-0.18778, -0.20534, … -1.025089]","[0.000066, 0.000069, … 0.000024]","[0.69373, 0.019801, … 72.28645]","[4.6339e-7, 5.3490e-7, … 0.000038]","[0.517101, 0.371513, … 34.629909]","[0.006786, -0.001167, … 0.041612]","[0.005859, 0.005054, … 0.028802]","[-0.000567, 0.000103, … -0.052593]","[0.598939, -0.085768, … 50.032721]","[-0.00049, -0.000446, … -0.036402]","[1.418214, 0.882783, … -0.412105]"
"[-0.003781, -0.001309, … 0.004499]","[7.429633, 7.616174, … 0.77588]","[0.00172, 0.001697, … 0.001258]","[0.261928, -0.016191, … -5.434772]","[0.339893, 0.338479, … -0.232213]","[-0.220234, -0.075214, … 0.191679]","[0.000014, 0.000002, … 0.00002]","[55.199439, 58.006113, … 0.60199]","[0.000003, 0.000003, … 0.000002]","[0.068606, 0.000262, … 29.53675]","[-0.028095, -0.009968, … 0.003491]","[-0.00099, 0.000021, … -0.024451]","[0.01278, 0.012927, … 0.000976]","[1.946029, -0.123316, … -4.216733]","[0.000451, -0.000027, … -0.006839]","[-0.667944, -0.804122, … 0.820629]"
"[-0.008102, -0.007577, … 0.016667]","[-13.405475, -12.142975, … 11.825626]","[-0.007065, -0.007211, … -0.00118]","[3.432059, 2.927974, … 3.577291]","[0.150215, 0.17083, … -0.121867]","[1.938205, 1.85015, … -0.665716]","[0.000066, 0.000057, … 0.000278]","[179.706773, 147.451835, … 139.845423]","[0.00005, 0.000052, … 0.000001]","[11.779026, 8.57303, … 12.797011]","[0.108611, 0.092011, … 0.1971]","[-0.027807, -0.022186, … 0.059623]","[0.094714, 0.087568, … -0.01395]","[-46.008377, -35.55431, … 42.303705]","[-0.024249, -0.021115, … -0.00422]","[0.206686, 1.034311, … -2.368951]"
"[-0.000705, -0.003476, … 0.004359]","[7.18851, 7.255315, … -1.118738]","[0.011553, 0.011468, … 0.001921]","[-0.966581, 0.23619, … 0.077109]","[0.138843, 0.132789, … -0.237516]","[-0.275857, -1.349696, … 0.283456]","[4.9735e-7, 0.000012, … 0.000019]","[51.674681, 52.6396, … 1.251575]","[0.000133, 0.000132, … 0.000004]","[0.934278, 0.055786, … 0.005946]","[-0.00507, -0.025219, … -0.004877]","[0.000682, -0.000821, … 0.000336]","[0.083046, 0.083205, … -0.002149]","[-6.948275, 1.713632, … -0.086265]","[-0.011167, 0.002709, … 0.000148]","[-0.29442, 0.846611, … 0.437807]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"[0.00273, -0.002679, … -0.012394]","[-15.816265, -15.520084, … -10.815311]","[0.001214, 0.000966, … 0.00499]","[2.764722, 4.059413, … -10.471988]","[0.402718, 0.40251, … -0.385353]","[0.112257, -0.087583, … -2.093855]","[0.000007, 0.000007, … 0.000154]","[250.154227, 240.873016, … 116.970942]","[0.000001, 9.3240e-7, … 0.000025]","[7.643688, 16.478837, … 109.662526]","[-0.043186, 0.041576, … 0.13404]","[0.007549, -0.010875, … 0.129785]","[-0.019205, -0.014986, … -0.053966]","[-43.727574, -63.002439, … 113.257799]","[0.003357, 0.00392, … -0.052253]","[-1.344711, -0.787466, … -0.490276]"
"[0.00382, 0.001066, … -0.010784]","[-6.111776, -6.408017, … 8.794863]","[0.002738, 0.002671, … 0.0023

In [30]:
import random
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchmetrics.regression import R2Score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed_val = 0

torch.manual_seed(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)

In [31]:
df = df.sample(n = len(df), with_replacement = False, shuffle = True, seed = seed_val)
df.head()

psi_e,b_e,psi_plus,b_plus,u_list,r_list,k_e_psi_e_list,k_e_b_e_list,k_e_psi_plus_list,k_e_b_plus_list,heat_flux_psi_e_b_e_list,heat_flux_psi_e_b_plus_list,b_e_psi_plus_list,b_e_b_plus_list,psi_plus_b_plus_list,eta_list
list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64]
"[0.008165, 0.007811, … -0.00415]","[1.161269, 0.515833, … -4.169906]","[-0.005056, -0.005316, … 0.00106]","[4.517383, 3.786941, … -5.237723]","[0.409687, 0.395351, … -0.282314]","[-1.397784, -1.405943, … -0.148894]","[0.000067, 0.000061, … 0.000017]","[1.348545, 0.266084, … 17.388118]","[0.000026, 0.000028, … 0.000001]","[20.406752, 14.340923, … 27.433743]","[0.009481, 0.004029, … 0.017307]","[0.036882, 0.029578, … 0.021739]","[-0.005872, -0.002742, … -0.004418]","[5.245896, 1.953431, … 21.840814]","[-0.022841, -0.020132, … -0.005549]","[-0.807688, -0.07953, … 0.030895]"
"[-0.011824, -0.011355, … 0.003199]","[-4.186379, -2.491497, … -3.204032]","[0.006693, 0.006277, … -0.000429]","[4.875028, 5.882524, … 2.504088]","[0.441348, 0.418674, … -0.285253]","[-2.679435, -2.413194, … -0.04646]","[0.00014, 0.000129, … 0.00001]","[17.525773, 6.207555, … 10.265824]","[0.000045, 0.000039, … 1.8400e-7]","[23.765896, 34.60409, … 6.270459]","[0.0495, 0.028291, … -0.010249]","[-0.057643, -0.066796, … 0.00801]","[-0.028018, -0.015638, … 0.001374]","[-20.408716, -14.656289, … -8.023181]","[0.032627, 0.036923, … -0.001074]","[-0.535052, 0.313884, … 0.504816]"
"[-0.000806, -0.001556, … 0.004717]","[-5.784642, -5.501922, … 17.14818]","[0.002485, 0.002565, … 0.002748]","[-1.738909, -1.057193, … -5.364035]","[0.341562, 0.338891, … -0.618841]","[-0.067815, -0.13513, … 0.438858]","[6.4948e-7, 0.000002, … 0.000022]","[33.462085, 30.271141, … 294.060086]","[0.000006, 0.000007, … 0.000008]","[3.023804, 1.117658, … 28.772874]","[0.004662, 0.008559, … 0.080892]","[0.001401, 0.001645, … -0.025304]","[-0.014376, -0.014115, … 0.047117]","[10.058966, 5.816594, … -91.983443]","[-0.004322, -0.002712, … -0.014738]","[0.396507, 1.095143, … 1.236072]"
"[-0.00614, -0.007678, … 0.023021]","[4.85624, 5.622175, … -10.821854]","[0.002244, 0.002065, … 0.002439]","[2.410232, 2.342461, … 9.4404]","[0.311213, 0.306687, … -0.564082]","[-0.466578, -0.536804, … 1.90096]","[0.000038, 0.000059, … 0.00053]","[23.583065, 31.608849, … 117.11252]","[0.000005, 0.000004, … 0.000006]","[5.809218, 5.487124, … 89.121156]","[-0.029817, -0.043165, … -0.249134]","[-0.014799, -0.017985, … 0.217331]","[0.010899, 0.01161, … -0.026392]","[11.704664, 13.169726, … -102.162631]","[0.005409, 0.004837, … 0.023023]","[-1.483728, -3.483646, … 1.314998]"
"[-0.005218, -0.00337, … 0.016261]","[-5.447419, -4.567568, … -7.979026]","[0.004133, 0.003846, … -0.001517]","[3.271144, 4.149766, … -3.486308]","[0.459821, 0.453844, … -0.31489]","[-0.730238, -0.438824, … -0.835176]","[0.000027, 0.000011, … 0.000264]","[29.674374, 20.862681, … 63.664857]","[0.000017, 0.000015, … 0.000002]","[10.700381, 17.220554, … 12.154343]","[0.028426, 0.015394, … -0.12975]","[-0.01707, -0.013986, … -0.056692]","[-0.022514, -0.017565, … 0.012103]","[-17.819291, -18.954338, … 27.817341]","[0.01352, 0.015958, … 0.005288]","[-0.995327, 1.399349, … 1.078835]"


### Train-Val-Test Split (70-20-10)

1. Split based on 1500 time-series samples (Vertically)
2. Split each sample to train-val-test (Horizontally)

Vertically for now!

#### How about 5-Fold Cross-Validation? Stratified 5F?

Just a single split for now!

In [32]:
df_train = df[:int(len(df) * 0.7)]
df_val = df[int(len(df) * 0.7):int(len(df) * 0.9)]
df_test = df[int(len(df) * 0.9):]

In [33]:
print(f"df_train shape: {df_train.shape}")
print(f"df_val shape: {df_val.shape}")
print(f"df_test shape: {df_test.shape}")

df_train shape: (1050, 16)
df_val shape: (300, 16)
df_test shape: (150, 16)


### Normalization

Important! Moving average must be implemented!

During training you shouldn't have access to future tokens for normalization! --> Data Leakage

For now just z-score on the whole train set for each row separately

In [34]:
df_train = df_train.select(
    (pl.col("*") - pl.col("*").list.mean()) / pl.col("*").list.std()
)

df_val = df_val.select(
    (pl.col("*") - pl.col("*").list.mean()) / pl.col("*").list.std()
)

df_test = df_test.select(
    (pl.col("*") - pl.col("*").list.mean()) / pl.col("*").list.std()
)

Probably a violin plot like the one here (https://www.tensorflow.org/tutorials/structured_data/time_series#normalize_the_data) is a good idea!

## Data Windowing

Must be super modular since we will run with diverse set of windows!

There is a helper function in Keras but I'm going full PyTorch! (`timeseries_dataset_from_array`)

In [35]:
def createWindowedDataframe(dataframe, stride, input_features, input_window_length, label_features, label_window_length):
    """
        Creates a windowed time-series dataset suitable for neural network (transformer) input

        dataframe: In time-series format where columns are features and rows are time-series steps (Polars exploded format!)
        stride: How many timesteps will each window move by. Equal for both input and label windows
        input_features: Names of the input features in a list
        input_window_length: Input window size in timesteps
        label_features: Names of the label features in a list
        label_window_length: label window size in timesteps
    """

    input_df = dataframe.select(
        pl.col(input_features)
    )
    input_df = input_df.select(
        numbers = pl.concat_list("*")   # List of all features
    )
    input_df = input_df[:-label_window_length].with_row_index("index").with_columns(    # Exluding the ending label window
        pl.col("index").cast(pl.Int64)
    )
    input_df = input_df.group_by_dynamic(   # index is considered for grouping
        index_column = "index",
        period = f"{input_window_length}i",
        every = f"{stride}i",
        closed = "left"
    ).agg(
        pl.col("numbers").alias("X"),
        pl.len().alias("seq_len")
    )
    input_df = input_df.filter(
        pl.col("seq_len") == input_window_length
    )
    input_df = input_df.select(
        pl.exclude(["index", "seq_len"])
    )


    label_df = dataframe.select(
        pl.col(label_features)
    )
    label_df = label_df.select(
        numbers = pl.concat_list("*")
    )
    label_df = label_df[input_window_length:].with_row_index("index").with_columns(
        pl.col("index").cast(pl.Int64)
    )
    label_df = label_df.group_by_dynamic(
        index_column = "index",
        period = f"{label_window_length}i",
        every = f"{stride}i",
        closed = "left"
    ).agg(
        pl.col("numbers").alias("Y"),
        pl.len().alias("seq_len")
    )
    label_df = label_df.filter(
        (pl.col("seq_len") == label_window_length)
    )
    label_df = label_df.select(
        pl.exclude(["index", "seq_len"])
    )

    return pl.concat([input_df, label_df], how = "horizontal")    

In [36]:
df_train = df_train.explode("*")

timeseries_df_train = pl.DataFrame()

for i in range(0, len(df_train), num_single_sample_timesteps):
    temp_df = createWindowedDataframe(
        dataframe = df_train[i:i + num_single_sample_timesteps],
        stride = window_stride,
        input_features = input_features,
        input_window_length = input_window_length,
        label_features = label_features,
        label_window_length = label_window_length
    )

    if(temp_df.is_empty()):
        timeseries_df_train = temp_df
    else:
        timeseries_df_train = pl.concat([timeseries_df_train, temp_df], how = "vertical")

timeseries_df_train

X,Y
list[list[f64]],list[list[f64]]
"[[0.695083, 0.137336, … -0.861463], [0.664438, 0.063593, … -0.113409], … [-0.353445, -1.241116, … -0.120229]]","[[0.4784, 0.629487], [0.504186, 0.547961], … [0.616578, 1.016295]]"
"[[0.664438, 0.063593, … -0.113409], [0.374989, 0.014885, … 0.871593], … [-0.545004, -1.175284, … -0.008268]]","[[0.504186, 0.547961], [0.533492, 1.032434], … [0.655694, 0.843847]]"
"[[0.374989, 0.014885, … 0.871593], [0.267195, -0.016321, … -2.005369], … [-0.501031, -1.085653, … -0.675027]]","[[0.533492, 1.032434], [0.575462, 1.06501], … [0.685162, 0.660106]]"
"[[0.267195, -0.016321, … -2.005369], [0.6086, -0.078537, … -0.282488], … [-1.043819, -0.986868, … -1.300909]]","[[0.575462, 1.06501], [0.616578, 1.016295], … [0.708982, 0.468925]]"
"[[0.6086, -0.078537, … -0.282488], [0.777913, -0.17832, … 0.509684], … [-1.162289, -0.826279, … 0.09535]]","[[0.616578, 1.016295], [0.655694, 0.843847], … [0.731119, 0.598273]]"
…,…
"[[-0.369838, 1.140922, … 0.323551], [-0.288735, 1.184916, … 0.215531], … [1.407745, -1.022153, … 0.393024]]","[[-1.029058, -0.186108], [-1.032003, 0.000375], … [-1.017156, 0.273713]]"
"[[-0.288735, 1.184916, … 0.215531], [-0.354677, 1.223125, … -0.195021], … [1.371097, -1.199764, … -0.153786]]","[[-1.032003, 0.000375], [-1.03029, 0.128355], … [-1.00965, 0.247844]]"
"[[-0.354677, 1.223125, … -0.195021], [-0.270865, 1.254407, … -0.743047], … [1.147891, -1.350696, … 1.019138]]","[[-1.03029, 0.128355], [-1.024585, 0.283519], … [-1.000976, 0.247122]]"


In [37]:
df_val = df_val.explode("*")

timeseries_df_val = pl.DataFrame()

for i in range(0, len(df_val), num_single_sample_timesteps):
    temp_df = createWindowedDataframe(
        dataframe = df_val[i:i + num_single_sample_timesteps],
        stride = window_stride,
        input_features = input_features,
        input_window_length = input_window_length,
        label_features = label_features,
        label_window_length = label_window_length
    )

    if(temp_df.is_empty()):
        timeseries_df_val = temp_df
    else:
        timeseries_df_val = pl.concat([timeseries_df_val, temp_df], how = "vertical")

timeseries_df_val

X,Y
list[list[f64]],list[list[f64]]
"[[0.180821, -0.188892, … -0.635586], [0.44814, -0.28685, … 1.669302], … [-0.658668, 1.039817, … -0.100583]]","[[0.885574, -0.373217], [0.870486, -0.305592], … [0.854747, 0.307443]]"
"[[0.44814, -0.28685, … 1.669302], [0.283726, -0.381432, … -0.958633], … [-0.633831, 1.123013, … 0.616437]]","[[0.870486, -0.305592], [0.85899, -0.202885], … [0.865868, 0.340771]]"
"[[0.283726, -0.381432, … -0.958633], [-0.182556, -0.415901, … -1.055087], … [-0.517644, 1.185055, … -0.064472]]","[[0.85899, -0.202885], [0.850945, -0.069844], … [0.876372, 0.492397]]"
"[[-0.182556, -0.415901, … -1.055087], [-0.45825, -0.388286, … 0.284904], … [-0.356269, 1.222506, … 1.044002]]","[[0.850945, -0.069844], [0.854747, 0.307443], … [0.89641, 0.602895]]"
"[[-0.45825, -0.388286, … 0.284904], [-0.659682, -0.337871, … -2.496378], … [-0.163539, 1.237487, … -0.566655]]","[[0.854747, 0.307443], [0.865868, 0.340771], … [0.92024, 0.696814]]"
…,…
"[[0.558747, 0.972802, … -0.375484], [1.056876, 0.8845, … 1.385156], … [-1.295997, -0.934619, … -0.033141]]","[[-1.072049, -2.114205], [-1.150213, -2.058439], … [-1.381632, -2.138535]]"
"[[1.056876, 0.8845, … 1.385156], [1.110417, 0.757332, … -0.973234], … [-1.867468, -0.629667, … 0.074526]]","[[-1.150213, -2.058439], [-1.221091, -1.999801], … [-1.445519, -1.575551]]"
"[[1.110417, 0.757332, … -0.973234], [1.491491, 0.591659, … -1.271651], … [-1.699596, -0.308389, … 0.426414]]","[[-1.221091, -1.999801], [-1.300293, -2.11688], … [-1.506044, -2.016674]]"


In [38]:
df_test = df_test.explode("*")

timeseries_df_test = pl.DataFrame()

for i in range(0, len(df_test), num_single_sample_timesteps):
    temp_df = createWindowedDataframe(
        dataframe = df_test[i:i + num_single_sample_timesteps],
        stride = window_stride,
        input_features = input_features,
        input_window_length = input_window_length,
        label_features = label_features,
        label_window_length = label_window_length
    )

    if(temp_df.is_empty()):
        timeseries_df_test = temp_df
    else:
        timeseries_df_test = pl.concat([timeseries_df_test, temp_df], how = "vertical")

timeseries_df_test

X,Y
list[list[f64]],list[list[f64]]
"[[0.143977, -0.613181, … -1.168601], [0.140459, -0.652986, … -0.109782], … [-0.22139, 1.900679, … 0.775233]]","[[0.399958, 0.305306], [0.408099, 0.313849], … [0.495336, 1.406211]]"
"[[0.140459, -0.652986, … -0.109782], [-0.205712, -0.657308, … -1.495147], … [0.331293, 1.870861, … 0.130115]]","[[0.408099, 0.313849], [0.426035, 0.740275], … [0.54421, 1.616953]]"
"[[-0.205712, -0.657308, … -1.495147], [-0.220805, -0.652731, … -0.010263], … [0.322671, 1.791194, … -1.268209]]","[[0.426035, 0.740275], [0.455748, 1.045916], … [0.590613, 1.5148]]"
"[[-0.220805, -0.652731, … -0.010263], [-0.411076, -0.61671, … 1.260905], … [0.794697, 1.667755, … -2.717769]]","[[0.455748, 1.045916], [0.495336, 1.406211], … [0.639615, 1.619529]]"
"[[-0.411076, -0.61671, … 1.260905], [-0.572723, -0.555823, … 0.413349], … [1.082893, 1.486055, … -2.828508]]","[[0.495336, 1.406211], [0.54421, 1.616953], … [0.684197, 1.631071]]"
…,…
"[[1.514743, 0.655541, … 0.721408], [1.47886, 0.33156, … 0.351551], … [-0.382596, 0.539714, … 0.82597]]","[[-1.754344, -0.003333], [-1.740474, 0.533514], … [-1.605575, 1.638573]]"
"[[1.47886, 0.33156, … 0.351551], [1.731567, -0.014363, … -0.878838], … [-0.033768, 0.720551, … 1.487962]]","[[-1.740474, 0.533514], [-1.711112, 0.969776], … [-1.536435, 1.965371]]"
"[[1.731567, -0.014363, … -0.878838], [1.633059, -0.364852, … -1.850758], … [0.228498, 0.809912, … -2.224276]]","[[-1.711112, 0.969776], [-1.6672, 1.45526], … [-1.453969, 2.790816]]"


**Some visualization here is good to do a sanity-check on the dataset before sending it to the model!!!!!!!!!!**

In [17]:
class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, dropout, max_len):
        super().__init__()
        self.dropout = torch.nn.Dropout(dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype = torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [18]:
class TimeSeriesTransformer(torch.nn.Module):
    def __init__(self, input_dim, output_dim, d_model, num_head, num_encoder_layers, num_decoder_layers, positional_encoding_max_len, position_wise_ffn_dim, dropout):
        super().__init__()
        self.input_proj = torch.nn.Linear(input_dim, d_model)
        self.output_proj = torch.nn.Linear(output_dim, d_model)
        
        self.pos_encoder = PositionalEncoding(
            d_model = d_model,
            dropout = dropout,
            max_len = positional_encoding_max_len
        )

        self.transformer = torch.nn.Transformer(
            d_model = d_model,
            nhead = num_head,
            num_encoder_layers = num_encoder_layers,
            num_decoder_layers = num_decoder_layers,
            dropout = dropout,
            dim_feedforward = position_wise_ffn_dim,
            batch_first = True
        )

        self.final_proj = torch.nn.Linear(d_model, output_dim)

    def forward(self, src, tgt, tgt_mask = None):
        src = self.input_proj(src)
        src = self.pos_encoder(src)
        
        tgt = self.output_proj(tgt)
        tgt = self.pos_encoder(tgt)
        
        if(tgt_mask is None):
            tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)

        decoder_out = self.transformer(src, tgt, tgt_mask = tgt_mask)

        return self.final_proj(decoder_out)

In [39]:
df_train = timeseries_df_train
df_val = timeseries_df_val
df_test = timeseries_df_test

%reset_selective -f "^timeseries_df_train$"
%reset_selective -f "^timeseries_df_val$"
%reset_selective -f "^timeseries_df_test$"
%reset_selective -f "^df$"

In [20]:
df_train = TensorDataset(
    torch.Tensor(df_train["X"]),
    torch.Tensor(df_train["Y"])
)
data_loader_train = DataLoader(
    df_train,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 10
)

df_val = TensorDataset(
    torch.Tensor(df_val["X"]),
    torch.Tensor(df_val["Y"])
)
data_loader_val = DataLoader(
    df_val,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 10
)

model = TimeSeriesTransformer(
    input_dim = len(input_features),
    output_dim = len(label_features),
    d_model = embedding_dim,
    num_head = num_attention_head,
    num_encoder_layers = num_encoder_layers,
    num_decoder_layers = num_encoder_layers,
    positional_encoding_max_len = positional_encoding_max_len,
    position_wise_ffn_dim = position_wise_nn_dim,
    dropout = dropout
).to(device)

print(f"Number of trainable parameters in the model: {sum(p.numel() for p in model.parameters() if p.requires_grad)}\n")

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = learning_rate
)

train_r2 = R2Score(multioutput = "uniform_average")
val_r2 = R2Score(multioutput = "uniform_average")

overfit_count = 0

for epoch in range(epochs):
    ################################################## TRAINING ##################################################
    model.train()
    epoch_train_loss = 0.0
    train_progress_bar = tqdm(
        data_loader_train,
        desc = f"Epoch {epoch + 1}/{epochs}"
    )
    
    for batch_x, batch_y in train_progress_bar:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        decoder_input = torch.zeros_like(batch_y)
        decoder_input[:, 1:] = batch_y[:, :-1]
        decoder_input[:, 0] = 0    # Adding bos token

        optimizer.zero_grad()
        output = model(batch_x, decoder_input)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()
        train_r2.update(
            output.view(output.shape[0], -1),    # Flatten (batch_size, timestep * num_feature)
            batch_y.view(batch_y.shape[0], -1)
        )
        train_progress_bar.set_postfix({
            "train_loss": f"{loss.item():.6f}"
        })

    avg_train_loss = epoch_train_loss / len(data_loader_train)
    epoch_train_r2 = train_r2.compute()
    train_r2.reset()
    print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {avg_train_loss:.6f}, Train R2: {epoch_train_r2:.6f}")


    ################################################# VALIDATION #################################################
    model.eval()
    epoch_val_loss = 0.0
    val_progress_bar = tqdm(
        data_loader_val,
        desc = f"Epoch {epoch + 1}/{epochs}"
    )

    with torch.no_grad():
        for batch_x, batch_y in val_progress_bar:
            # No teacher forcing in inference: Output at each timestep is feeded back to the decoder as input in the next timestep
            # Attention mask grows as timesteps pass
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            outputs = []
            decoder_input = torch.zeros(batch_y.shape[0], 1, len(label_features)).to(device)
            for t in range(batch_y.shape[1]):
                tgt_mask = model.transformer.generate_square_subsequent_mask(decoder_input.shape[1]).to(device)
                out = model(
                    batch_x,
                    decoder_input,
                    tgt_mask = tgt_mask
                )
                next_step = out[:, -1:, :]
                outputs.append(next_step)
                decoder_input = torch.cat([decoder_input, next_step], dim = 1)

            pred = torch.cat(outputs, dim = 1)
            loss = criterion(pred, batch_y)
            epoch_val_loss += loss.item()
            val_r2.update(
                pred.view(pred.shape[0], -1),    # Flatten (batch_size, timestep * num_feature)
                batch_y.view(batch_y.shape[0], -1)
            )
            val_progress_bar.set_postfix({
                "val_loss": f"{loss.item():.6f}"
            })

        avg_val_loss = epoch_val_loss / len(data_loader_val)
        epoch_val_r2 = val_r2.compute()
        val_r2.reset()
        print(f"Epoch [{epoch + 1}/{epochs}], Val Loss: {avg_val_loss:.6f}, Val R2: {epoch_val_r2:.6f}\n")
    

    if(avg_val_loss - avg_train_loss > 0.05):
        overfit_count += 1
        print(f"Possible Overfitting!!! {overfit_count}/3\n")
        if(overfit_count == 3):
            print("Training Stopped!!!")
            break

Number of trainable parameters in the model: 31042



Epoch 1/20: 100%|██████████| 32025/32025 [08:18<00:00, 64.24it/s, train_loss=0.039596]


Epoch [1/20], Train Loss: 0.097798, Train R2: 0.902750


Epoch 1/20: 100%|██████████| 9150/9150 [02:20<00:00, 64.99it/s, val_loss=0.059384]


Epoch [1/20], Val Loss: 0.110165, Val R2: 0.890549



...

Epoch [19/20], Val Loss: 0.079789, Val R2: 0.920768



Epoch 20/20: 100%|██████████| 32025/32025 [08:23<00:00, 63.58it/s, train_loss=0.021876]


Epoch [20/20], Train Loss: 0.031220, Train R2: 0.968995


Epoch 20/20: 100%|██████████| 9150/9150 [02:27<00:00, 62.07it/s, val_loss=0.147994]


Epoch [20/20], Val Loss: 0.079366, Val R2: 0.921186



In [40]:
################################################## TESTING ##################################################
df_test = TensorDataset(
    torch.Tensor(df_test["X"]),
    torch.Tensor(df_test["Y"])
)
data_loader_test = DataLoader(
    df_test,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 10
)

model.eval()
test_loss = 0.0
test_progress_bar = tqdm(
    data_loader_val
)

test_r2 = R2Score(multioutput = "uniform_average")

with torch.no_grad():
    for batch_x, batch_y in test_progress_bar:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        outputs = []
        decoder_input = torch.zeros(batch_y.shape[0], 1, len(label_features)).to(device)
        for t in range(batch_y.shape[1]):
            tgt_mask = model.transformer.generate_square_subsequent_mask(decoder_input.shape[1]).to(device)
            out = model(
                batch_x,
                decoder_input,
                tgt_mask = tgt_mask
            )
            next_step = out[:, -1:, :]
            outputs.append(next_step)
            decoder_input = torch.cat([decoder_input, next_step], dim = 1)

        pred = torch.cat(outputs, dim=1)
        loss = criterion(pred, batch_y)
        test_loss += loss.item()
        test_progress_bar.set_postfix({
            "batch_test_loss": f"{loss.item():.6f}"
        })

        test_r2.update(
            pred.view(pred.shape[0], -1),
            batch_y.view(batch_y.shape[0], -1)
        )

    final_test_loss = test_loss / len(data_loader_test)
    final_test_r2 = test_r2.compute()
    test_r2.reset()
    print(f"Final Test Loss: {final_test_loss:.6f}, Final Test R2: {final_test_r2:.6f}")

100%|██████████| 9150/9150 [02:20<00:00, 65.29it/s, batch_test_loss=0.085025] 

Final Test Loss: 0.158731, Final Test R2: 0.921185


In [22]:
torch.save(model, model_path)